# BGPピア閉塞デモ

# [1]Inventoryの情報を定義

In [1]:
#inventory
[NS2250]
XXX.XXX.XXX.XXX

[NS2250:vars]
ansible_ssh_user=admin
ansible_ssh_pass=XXXX
ansible_connection=network_cli
ansible_network_os=smartcs
ansible_command_timeout=60

# [2]環境変数(Timeout値、sshkeyのチェック無効化の設定)

In [2]:
#ansible.cfg
[persistent_connection]
command_timeout = 360
connect_timeout = 360
connect_retry_timeout = 360

[defaults]
host_key_checking=False

# [3]varsファイルの定義(対象NW機器の情報)

In [3]:
#vars var
iosxr:
  tty: 1
  user: admin
  password: XXXX
localas: 64540
neighbor: 172.16.1.10

# [4]Playbookの実行

In [4]:
#play
hosts: NS2250
gather_facts: no
vars_files:
    - var

# [5]Taskの実行(ログインプロンプトに戻ってない場合の処理)

In [5]:
#task
name: test
smartcs_tty_command:
  tty: "{{ iosxr.tty }}"
  nl: cr
  cmd_timeout: 5
  sendchar:
  - __NL__
  - exit
  cmd_timeout_onfail: no
register: result

TASK [test] ********************************************************************


# [6] [5]の実行結果表示

In [6]:
debug: var=result["stdout"]

TASK [debug] *******************************************************************
ok: [XXX.XXX.XXX.XXX] => {
    "result[\"stdout\"]": [
        "", 
        "", 
        "", 
        "", 
        "User Access Verification\n\n\n\nUsername: \nError: Timeout.", 
        "exit\n\n\nPassword: \nError: Timeout."
    ]
}


# [7]Taskの実行(ログイン)

In [7]:
#task
name: ios-xr login
smartcs_tty_command:
  tty:  "{{ iosxr.tty }}"
  nl: cr
  cmd_timeout: 10
  recvchar_regex:
  - "Username:"
  - "Password:"
  - "#"
  sendchar:
  - __NL__
  - "{{ iosxr.user }}"
  - "{{ iosxr.password }}"
  - terminal length 0
  cmd_timeout_onfail: no
register: result

TASK [ios-xr login] ************************************************************


# [8] [7]の実行結果表示

In [8]:
name: show task result
debug: var=result["stdout_lines"]

TASK [show task result] ********************************************************
ok: [XXX.XXX.XXX.XXX] => {
    "result[\"stdout_lines\"]": [
        [
            ""
        ], 
        [
            ""
        ], 
        [
            ""
        ], 
        [
            "waitregex> Username:"
        ], 
        [
            "waitregex> Password:"
        ], 
        [
            "waitregex> #"
        ], 
        [
            ""
        ], 
        [
            "RP/0/RSP0/CPU0:Mar  7 17:29:00.875 : exec[65777]: %SECURITY-login-4-AUTHEN_FAILED : Failed authentication attempt by user 'exit' from 'console' on 'con0_RSP0_CPU0' ", 
            "", 
            "", 
            "", 
            "User Access Verification", 
            "", 
            "", 
            "", 
            "Username:"
        ], 
        [
            "admin", 
            "", 
            "", 
            "Password:"
        ], 
        [
            "RP/0/RSP0/CPU0:Hyoka-XXXX#"
        ], 
        [
  

# [9] Taskの実行(show bgp summaryの投入)

In [9]:
#task
name: show bgp summary
smartcs_tty_command:
  tty: "{{ iosxr.tty }}"
  nl: cr
  cmd_timeout: 60
  recvchar_regex:
  - "#"
  sendchar:
  - "show bgp summary"
  cmd_timeout_onfail: no
register: result

TASK [show bgp summary] ********************************************************


# [10] [9]の実行結果 UpしているNeighborが存在することを確認

In [10]:
name: show task result
debug: var=result["stdout_lines"][-1]

TASK [show task result] ********************************************************
ok: [XXX.XXX.XXX.XXX] => {
    "result[\"stdout_lines\"][-1]": [
        "show bgp summary", 
        "", 
        "", 
        "Thu Mar  7 17:29:34.492 JST", 
        "", 
        "BGP router identifier 172.16.1.2, local AS number 64540", 
        "", 
        "BGP generic scan interval 60 secs", 
        "", 
        "BGP table state: Active", 
        "", 
        "Table ID: 0xe0000000   RD version: 8", 
        "", 
        "BGP main routing table version 8", 
        "", 
        "BGP scan interval 60 secs", 
        "", 
        "", 
        "", 
        "BGP is operating in STANDALONE mode.", 
        "", 
        "", 
        "", 
        "", 
        "", 
        "Process       RcvTblVer   bRIB/RIB   LabelVer  ImportVer  SendTblVer  StandbyVer", 
        "", 
        "Speaker               8          8          8          8           8           8", 
        "", 
        "", 
        "", 
        

# [11] configureモードに変更

In [11]:
#task
name: configure
smartcs_tty_command:
  tty: "{{ iosxr.tty }}"
  nl: cr
  cmd_timeout: 5
  recvchar_regex:
  - "#"
  - "#"
  sendchar:
  - configure
  cmd_timeout_onfail: no
register: result

TASK [configure] ***************************************************************


# [12] [11]の実行結果

In [12]:
name: show task result
debug: var=result["stdout_lines"]

TASK [show task result] ********************************************************
ok: [XXX.XXX.XXX.XXX] => {
    "result[\"stdout_lines\"]": [
        [
            ""
        ], 
        [
            ""
        ], 
        [
            ""
        ], 
        [
            "waitregex> #"
        ], 
        [
            "waitregex> #"
        ], 
        [
            ""
        ], 
        [
            "configure", 
            "", 
            "", 
            "Thu Mar  7 17:29:54.998 JST", 
            "", 
            "RP/0/RSP0/CPU0:Hyoka-XXXX(config)#"
        ]
    ]
}


# [13] bgp neigborのshutdown実行

In [13]:
#task
name: |
  - "router bgp {{ localas }}"
  - "neighbor {{ neighbor }} shutdown"
smartcs_tty_command:
  tty: "{{ iosxr.tty }}"
  nl: cr
  cmd_timeout: 5
  recvchar_regex:
  - "#"
  - "#"
  - "#"
  - "#"
  - "#"
  sendchar:
  - "router bgp {{ localas }}"
  - "neighbor {{ neighbor }} shutdown"
  - commit
  - exit
  - exit
  cmd_timeout_onfail: no
register: result

TASK [- "router bgp 64540"
- "neighbor 172.16.1.10 shutdown"
] *****************


# [14] [13]の実行結果

In [14]:
name: show task result
debug: var=result["stdout_lines"]

TASK [show task result] ********************************************************
ok: [XXX.XXX.XXX.XXX] => {
    "result[\"stdout_lines\"]": [
        [
            ""
        ], 
        [
            ""
        ], 
        [
            ""
        ], 
        [
            "waitregex> #"
        ], 
        [
            "waitregex> #"
        ], 
        [
            "waitregex> #"
        ], 
        [
            "waitregex> #"
        ], 
        [
            "waitregex> #"
        ], 
        [
            ""
        ], 
        [
            "router bgp 64540", 
            "", 
            "", 
            "RP/0/RSP0/CPU0:Hyoka-XXXX(config-bgp)#"
        ], 
        [
            "neighbor 172.16.1.10 shutdown", 
            "", 
            "", 
            "RP/0/RSP0/CPU0:Hyoka-XXXX(config-bgp)#"
        ], 
        [
            "commit", 
            "", 
            "", 
            "Thu Mar  7 17:30:13.239 JST", 
            "", 
            "RP/0/RSP0/CPU0:Hyoka-XXXX(c

# [15] Taskの実行(show bgp summaryの投入)

In [15]:
#task
name: show bgp summary
smartcs_tty_command:
  tty: "{{ iosxr.tty }}"
  nl: cr
  cmd_timeout: 60
  recvchar_regex:
  - "#"
  sendchar:
  - "show bgp summary"
  cmd_timeout_onfail: no
register: result

TASK [show bgp summary] ********************************************************


# [16] [15]の実行結果 admin Idleしていることを確認

In [16]:
name: show task result
debug: var=result["stdout_lines"]

TASK [show task result] ********************************************************
ok: [XXX.XXX.XXX.XXX] => {
    "result[\"stdout_lines\"]": [
        [
            ""
        ], 
        [
            ""
        ], 
        [
            ""
        ], 
        [
            "waitregex> #"
        ], 
        [
            ""
        ], 
        [
            "show bgp summary", 
            "", 
            "", 
            "Thu Mar  7 17:30:35.258 JST", 
            "", 
            "BGP router identifier 172.16.1.2, local AS number 64540", 
            "", 
            "BGP generic scan interval 60 secs", 
            "", 
            "BGP table state: Active", 
            "", 
            "Table ID: 0xe0000000   RD version: 8", 
            "", 
            "BGP main routing table version 8", 
            "", 
            "BGP scan interval 60 secs", 
            "", 
            "", 
            "", 
            "BGP is operating in STANDALONE mode.", 
            "", 
           

# [17] Taskの実行(exit処理)

In [17]:
#task
name: Exit
smartcs_tty_command:
  tty: "{{ iosxr.tty }}"
  nl: cr
  cmd_timeout: 5
  recvchar_regex:
  - "#"
  sendchar:
  - __NL__
  - exit
  cmd_timeout_onfail: no
register: result

TASK [Exit] ********************************************************************


# [18] [17]の実行結果

In [18]:
name: show task result
debug: var=result["stdout_lines"]

TASK [show task result] ********************************************************
ok: [XXX.XXX.XXX.XXX] => {
    "result[\"stdout_lines\"]": [
        [
            ""
        ], 
        [
            ""
        ], 
        [
            ""
        ], 
        [
            "waitregex> #"
        ], 
        [
            ""
        ], 
        [
            "RP/0/RSP0/CPU0:Hyoka-XXXX#"
        ], 
        [
            "exit", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
            "", 
      